In [1]:
import numpy as np
from numpy import asarray
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
from tqdm import tqdm
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from xgboost import XGBRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [2]:
df = pd.read_csv('Dasatest_N0.csv')
df['Month'] = pd.to_datetime(df.Month , format = '%d/%m/%Y')
data = df.drop(['Month'], axis=1)
data.index = df.Month
cols = data.columns

In [3]:
pred = np.zeros([15,3915])

In [4]:
# preparing independent and dependent features
def prepare_data(timeseries_data, n_features):
	X, y =[],[]
	for i in range(len(timeseries_data)):
		# find the end of this pattern
		end_ix = i + n_features
		# check if we are beyond the sequence
		if end_ix > len(timeseries_data)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = timeseries_data[i:end_ix], timeseries_data[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)

In [5]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
# 	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols = list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
	# put it all together
	agg = concat(cols, axis=1)
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg.values  

In [6]:
def model_create(X, y):
    model = XGBRegressor(objective='reg:squarederror', n_estimators=10)
    model.fit(X, y)
    return model

In [7]:
def model_create_2(X, y):
    model = RandomForestRegressor(n_estimators=5)
    model.fit(X, y)
    return model

In [8]:
def model_create_3(X, y):
    model = AdaBoostRegressor(n_estimators=5)
    model.fit(X, y)
    return model

In [9]:
def model_create_4(X, y):
    model = GradientBoostingRegressor(n_estimators=5)
    model.fit(X, y)
    return model

In [10]:
def model_predict(model,X_input,n_steps,n_steps_out):   
    x_input = X_input
#     print(x_input)
    temp_input = list(x_input)
#     print(len(temp_input))
    lst_output=[]
    i = 0
    while(i < n_steps_out):
        
        if(len(temp_input)>n_steps):
            x_input=np.array(temp_input[1:])
#             print("{} day input {}".format(i,x_input))
            #print(x_input)
            x_input = x_input.reshape(n_steps)
            #print(x_input)
            yhat = model.predict(asarray([x_input]))
#             print("{} day output {}".format(i,yhat))
            temp_input.append(yhat[0])
            temp_input=temp_input[1:]
            #print(temp_input)
            lst_output.append(yhat[0])
            i=i+1
        else:
            x_input = x_input.reshape(n_steps)
            yhat = model.predict(asarray([x_input]))
#             print(yhat[0])
            temp_input.append(yhat[0])
            lst_output.append(yhat[0])
            i=i+1

#     print(lst_output)
    return(np.array(lst_output))

In [11]:
def plot_graph(X,pred):
    time = np.arange(87)
    plt.plot(time[60:72],X)
    plt.plot(time[60:72],pred)
    plt.show()

In [12]:
prediction = np.zeros([15,3915])
prediction1 = np.zeros([15,3915])
sum1 = 0 

In [13]:
def model_sarakam(n_in,train_real,rem):
    train = series_to_supervised(train_real, n_in=n_in)
    # split into input and output columns
    trainX, trainy = train[:, :-1], train[:, -1]
#     print(trainX.shape)
    model = model_create_3(trainX, rem[-len(trainX):])
    X_input = data_total[:n_in].reshape((n_in))
    n_steps_out = len(trainX)
    pred = model_predict(model,X_input,n_in,n_steps_out)
    pred = np.array(pred)
    return pred,model

In [14]:
def model_sarakam_2(n_in,train_real,rem):
    train = series_to_supervised(train_real, n_in=n_in)
    # split into input and output columns
    trainX, trainy = train[:, :-1], train[:, -1]
#     print(trainX.shape)
    model = model_create_4(trainX, rem[-len(trainX):])
    X_input = data_total[:n_in].reshape((n_in))
    n_steps_out = len(trainX)
    pred = model_predict(model,X_input,n_in,n_steps_out)
    pred = np.array(pred)
    return pred,model

In [15]:
sum1 = 0
for i in tqdm(range(3915)):
    data_total = np.array(data.iloc[:,i])
    train_real = data_total
#     valid = data_total[60:]
    
    n_in_1 = 3
    pred1, model1 = model_sarakam(n_in_1,train_real,train_real)
    rem1 = train_real[-len(pred1):] - pred1
    
    n_in_2 = 6
    pred2, model2 = model_sarakam_2(n_in_2,train_real,rem1)
    rem2 = rem1[-len(pred2):] - pred2
    
    n_in_3 = 12
    pred3, model3 = model_sarakam(n_in_3,train_real,rem2)
    rem3 = rem2[-len(pred3):] - pred3
    
    n_in_4 = 18
    pred4, model4 = model_sarakam_2(n_in_4,train_real,rem3)
    rem4 = rem3[-len(pred4):] - pred4
    
    n_in_5 = 24
    pred5, model5 = model_sarakam(n_in_5,train_real,rem4)
    rem5 = rem4[-len(pred5):] - pred5
    
    n_in_6 = 36
    pred6, model6 = model_sarakam_2(n_in_6,train_real,rem5)
    rem6 = rem5[-len(pred6):] - pred6
    
    n_steps_out = 15
    
    X_input_6 = train_real[-n_in_6:]
    fpred_6 = model_predict(model6,X_input_6,n_in_6,n_steps_out)
    
    X_input_5 = train_real[-n_in_5:]
    fpred_5 = model_predict(model5,X_input_5,n_in_5,n_steps_out)
    
    X_input_4 = train_real[-n_in_4:]
    fpred_4 = model_predict(model4,X_input_4,n_in_4,n_steps_out)
    
    X_input_3 = train_real[-n_in_3:]
    fpred_3 = model_predict(model3,X_input_3,n_in_3,n_steps_out)
    
    X_input_2 = train_real[-n_in_2:]
    fpred_2 = model_predict(model2,X_input_2,n_in_2,n_steps_out)
    
    X_input_1 = train_real[-n_in_1:]
    fpred_1 = model_predict(model1,X_input_1,n_in_1,n_steps_out)
    
    fpred = (n_in_6*fpred_1+n_in_5*fpred_2+n_in_4*fpred_3+n_in_3*fpred_4+n_in_2*fpred_5+n_in_1*fpred_6)/(n_in_1+n_in_2+n_in_3+n_in_4+n_in_5+n_in_6)
    fpred1 = fpred_1 + fpred_2 + fpred_3 + fpred_4 + fpred_5 + fpred_6
    
    prediction[:,i] = fpred
    prediction1[:,i] = fpred1

100%|██████████████████████████████████████████████████████████████████████████████| 3915/3915 [07:59<00:00,  8.16it/s]


In [16]:
print(prediction)
print(prediction1)


[[0.1215175  0.10776868 0.20568516 ... 0.55880237 0.50200388 0.34850353]
 [0.07304932 0.11237204 0.11366369 ... 0.43863158 0.40192234 0.24291177]
 [0.09024438 0.17190465 0.1254152  ... 0.52961848 0.31610219 0.2454426 ]
 ...
 [0.08532892 0.12970861 0.10672356 ... 0.39070167 0.73827833 0.15680356]
 [0.09146262 0.1229471  0.09871554 ... 0.34726899 0.72300847 0.13395757]
 [0.09269322 0.12344921 0.10043264 ... 0.344225   0.65121164 0.13381916]]
[[0.54040025 0.47611042 0.93723858 ... 3.09166323 2.31841745 1.75715831]
 [0.28878277 0.47059708 0.49337689 ... 2.35594838 1.40598453 1.27007385]
 [0.41524069 0.67869253 0.59400777 ... 2.74428301 1.16168732 1.46762513]
 ...
 [0.32649965 0.54514964 0.46429362 ... 1.64333535 2.15301356 0.74194556]
 [0.32556282 0.46450722 0.31939996 ... 1.40445563 1.90106091 0.58246751]
 [0.33571522 0.45945003 0.37007319 ... 1.37934271 1.70361963 0.57790008]]


In [17]:
output = pd.DataFrame(prediction)
output.to_csv('submission_hybrid_mix_11.csv')
print("Your submission was successfully saved!")

Your submission was successfully saved!


In [18]:
output = pd.DataFrame(prediction1)
output.to_csv('submission_hybrid_mix_t1.csv')
print("Your submission was successfully saved!")

Your submission was successfully saved!
